# Train Using SageMaker Manifest File with Scikit-Learn Decision Tree Regressor

This is a sample Python program that trains a simple scikit-learn model on the California dataset using Manifest File.

for more details please refer the documentation: https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_S3DataSource.html


**This sample is provided for demonstration purposes, make sure to conduct appropriate testing if derivating this code for your own use-cases!**

In [ ]:
!pip install sagemaker -U -q

In [ ]:
import datetime
import time
import tarfile

import boto3
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing

sm_boto3 = boto3.client("sagemaker")
s3_boto3 = boto3.client('s3')
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
region = sess.boto_session.region_name
bucket = sess.default_bucket()  # this could also be a hard-coded bucket name

print(f"Using sagemaker version {sagemaker.__version__}")

## SageMaker Training

### Launching a training job with the Python SDK using Manifest File

In [ ]:
# We use the Estimator from the SageMaker Python SDK
from sagemaker.sklearn.estimator import SKLearn

sklearn_estimator = SKLearn(
        entry_point="scikit_learn_california.py",
        source_dir="code",
        framework_version="1.2-1",
        instance_type="ml.c5.xlarge",
        # keep_alive_period_in_seconds=3600,
        role=role,
        hyperparameters={"max_leaf_nodes": 30},
)

### Manifest File

A manifest might look like this: s3://bucketname/example.manifest

A manifest is an S3 object which is a JSON file consisting of an array of elements. The first element is a prefix which is followed by one or more suffixes. SageMaker appends the suffix elements to the prefix to get a full set of S3Uri. Note that the prefix must be a valid non-empty S3Uri that precludes users from specifying a manifest whose individual S3Uri is sourced from different S3 buckets.

The following code example shows a valid manifest format:
```

[ {"prefix": "s3://customer_bucket/some/prefix/"},

"relative/path/to/custdata-1",

"relative/path/custdata-2",

...

"relative/path/custdata-N"

]
```

This JSON is equivalent to the following S3Uri list:

- s3://customer_bucket/some/prefix/relative/path/to/custdata-1

- s3://customer_bucket/some/prefix/relative/path/custdata-2

- ...

- s3://customer_bucket/some/prefix/relative/path/custdata-N

The complete set of S3Uri in this manifest is the input data for the channel for this data source. The object that each S3Uri points to must be readable by the IAM role that SageMaker uses to perform tasks on your behalf.

Let's inspect our Manifest file

In [ ]:
manifest_file_path = "s3://aws-ml-blog/artifacts/scikit-learn-training-w-manifest/california-housing-manifest.json"

In [ ]:
import json

s3_response = s3_boto3.get_object(
        Bucket='aws-ml-blog',
        Key='artifacts/scikit-learn-training-w-manifest/california-housing-manifest.json'
    )
s3_object_body = s3_response.get('Body')
content_str = s3_object_body.read().decode()
json.loads(content_str)

You can see that there are three files. One file for train, one file for test, and one file for validation.

Now let's inspect the files in the bucket

In [ ]:
!aws s3 ls s3://aws-ml-blog/artifacts/scikit-learn-training-w-manifest/ --recursive

We can clearly see that there are many files for train, test, and validation.

During SageMaker raining job run, we expect to load only one file for train, one file for test, and one file for validation using the Manifest file we review previously.

In [ ]:
inputs = sagemaker.inputs.TrainingInput(
    s3_data=manifest_file_path, s3_data_type="ManifestFile"
)

In [ ]:
sklearn_estimator.fit(inputs)